In [1]:
import numpy as np
import scipy as sp
import torch
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas
import nibabel as nib
import sys
sys.path.insert(0,'../deepfly/')
import ft_parser as ft
from scipy.interpolate import interp1d
import sklearn
from sklearn.linear_model import LinearRegression
import torch
import pandas as pd
import os
import torch.nn as nn
import torch.optim as optim

2021-11-28 12:29:17.870641: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-28 12:29:17.870705: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Load in the data.

In [3]:
# Constant to scale the outputs by.
BEH_SCALE = 100

# Supervoxel data.
supervoxel_datapath = '/data/minseung/deepfly/supervoxel/non-interpolated/X_all_neural_z_vox_t_correct.npy'
supervoxel_data = np.load(supervoxel_datapath)
 
# Acquisition timestamps for neural data.
timestamps_path = '/data/minseung/deepfly/supervoxel/non-interpolated/X_all_neural_timestamps_z_t.npy'
X_all_neural_timestamps_z_t = np.load(timestamps_path)

# Forward velocity.
fwd_path = '/data/minseung/deepfly/behavior/20210525_all_beh.npy'
fwd = np.load(fwd_path)
fwd *= BEH_SCALE

# Rotational velocity, also scale b
rot_path = '/data/minseung/deepfly/behavior/20210525_all_beh_rot.npy'
rot = np.load(rot_path)
rot *= BEH_SCALE

# Behaviorial timestamps.
behavioral_timestamps_path = '/data/minseung/deepfly/behavior/20210525_all_beh_time.npy'
behavioral_timestamps = np.load(behavioral_timestamps_path)

## Resample the behavioral data to match the neural and behavioral sampling and write data to disk.

In [4]:
# Some constants
Z_AXIS_IND = 23 # a z-axis to use
TRAIN_FRAC = .90

# Grab the first and last time points from neural data.
first_tp = X_all_neural_timestamps_z_t[0,0] #from the top z layer
last_tp = X_all_neural_timestamps_z_t[-1,0] #from the top z layer

# Generate the same number of points to match neural data.
timepoints_to_pull = np.linspace(first_tp, last_tp, X_all_neural_timestamps_z_t.shape[0])

# Interpolate neural data to be time-aligned to the top z layer
supervoxel_interp = np.asarray([[np.interp(timepoints_to_pull, X_all_neural_timestamps_z_t[:,z], supervoxel_data[z,sv,:]) for sv in range(supervoxel_data.shape[1])] for z in range(int(supervoxel_data.shape[0]/2))])

# Reshape the data using only one of the Z axes.
n_z_slices,n_supervoxels,t = supervoxel_interp.shape
supervoxel_interp = supervoxel_interp[Z_AXIS_IND,...].reshape(1,n_supervoxels,t)

# Reshape the neural data so that all supervoxels are features.
supervoxel_interp = np.transpose(np.reshape(supervoxel_interp,(supervoxel_interp.shape[0] * supervoxel_interp.shape[1],supervoxel_interp.shape[2])))
  
# Downsample forward velocity.
behavior_interp_obj = interp1d(behavioral_timestamps, fwd, bounds_error = False)
Y = behavior_interp_obj(timepoints_to_pull)
Y = np.nan_to_num(Y)
Y_fwd = Y[:,np.newaxis]

# Now, downsample rotational velocity and stack them together in a matrix.
behavior_interp_obj = interp1d(behavioral_timestamps, rot, bounds_error = False)
Y = behavior_interp_obj(timepoints_to_pull)
Y = np.nan_to_num(Y)
Y_rot = Y[:,np.newaxis]
Y = np.c_[Y_fwd,Y_rot]

# Make a training and testing split and write this to disk.
np.random.seed(2021)
n = supervoxel_interp.shape[0]
inds_all = np.arange(0,n)
train_inds = np.sort(np.random.choice(inds_all,int(np.floor(TRAIN_FRAC * n)),
                                     replace=False))
test_inds = np.setdiff1d(inds_all,train_inds)

# Store the data into a dictionary and write to disk.
train_data = dict()
train_data['supervoxel_interp'] = supervoxel_interp[train_inds,:]
train_data['Y'] = Y[train_inds,:]

outdir = '/data/minseung/deepfly/preprocessed/'

if not os.path.isdir(outdir):
    os.makedirs(outdir)
    
fnameout = os.path.join(outdir,"train_data_layer_no_fwd_clip%s"%str(Z_AXIS_IND))
np.save(fnameout,train_data,allow_pickle=True)

# Same with the testing split.
test_data = dict()
test_data['supervoxel_interp'] = supervoxel_interp[test_inds,:]
test_data['Y'] = Y[test_inds,:]

outdir = '/data/minseung/deepfly/preprocessed/'

if not os.path.isdir(outdir):
    os.makedirs(outdir)
     
fnameout = os.path.join(outdir,"test_data_layer_no_fwd_clip%s"%str(Z_AXIS_IND))
np.save(fnameout,test_data,allow_pickle=True)